#COMMON READABILITY COMPETITION 

# Clean the Unstructured Language Data

In [1]:
#*Importing Main Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Reading the data
training_syntax= pd.read_csv('../input/commonlitreadabilityprize/train.csv')
testing_syntax=pd.read_csv('../input/commonlitreadabilityprize/test.csv')
save=pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

In [3]:
#explore the data: general shape od the df
print("shape of training_syntax",training_syntax.shape)
training_syntax.head(3)
training_syntax.tail(3)

shape of training_syntax (2834, 6)


,id,url_legal,license,excerpt,target,standard_error
2831,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2832,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128
2833,5b990ba77,https://en.wikibooks.org/wiki/Wikijunior:Biolo...,CC BY-SA 3.0,Animals are made of many cells. They eat thing...,0.300779,0.512379


 # Clean and turn the Unstructured Language Data to structured data structures(array, dictionary, etc)

In [4]:
#Importing NLTK librariries so that to clean the unstructured language data
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk import word_tokenize
from nltk import pos_tag
import spacy
english_spacy = spacy.load('en_core_web_lg')
from spacy import displacy
import string
import re
nltk.download('stopwords', quiet=True)
stopwords = nltk.corpus.stopwords.words('english')

#removing punctuations and stopwords from unstructured langugae data 
def stopwordsPunctuation_detacher(syntax):
    syntax = re.sub(r'[^\w\s]','',syntax)
    syntax = [lexUnit.lower() for lexUnit in syntax.lower().split() if lexUnit not in stopwords]
    syntax=' '.join(syntax)
    syntax.translate(str.maketrans('','', string.punctuation))
    english_sytax = english_spacy(syntax)
    new_syntax = ' '
    absence=0
    for lex_unit in english_sytax:
        if (lex_unit.is_stop == absence):
            new_syntax = new_syntax + ' ' + str(lex_unit)
    return new_syntax

#removing digits, webLinks from unstructured langugae data and making all words lower case.
def textCleaning_engine(syntax):
    syntax = syntax.lower()
    syntax = re.sub('\[.*?\]', '', syntax)
    syntax = re.sub('https?://\S+|www\.\S+', '', syntax)
    syntax = re.sub('<.*?>+', '', syntax)
    syntax = re.sub('[%s]' % re.escape(string.punctuation), '', syntax)
    syntax = re.sub('\n', '', syntax)
    syntax = re.sub('\w*\d\w*', '', syntax)
    clean_syntax = re.sub(r'\d+','',syntax)
    return clean_syntax
#Combining the two functions above in one function
def ready_sytaxt(syntax):
    syntax = syntax.lower()
    syntax = stopwordsPunctuation_detacher(syntax)
    syntax = textCleaning_engine(syntax)
    return syntax
#applying the function created above to the datasets so that to remove the unecessary lexical units(such as stopWOrds, punctuations, digits, links, ets)
training_syntax['excerpt_clean'] = training_syntax['excerpt'].apply(ready_sytaxt)
testing_syntax['excerpt_clean'] = testing_syntax['excerpt'].apply(ready_sytaxt) 


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [5]:
#storing the elements of training_syntax['excerpt_clean'] in a new container as dictionary keys and those elements' counts as dictionary values.
from collections import Counter
dictionaryKeys_container = Counter()
training_syntax['excerpt_clean'].str.lower().str.split().apply(dictionaryKeys_container.update)

# reducing a word to its word stem(roots of words) to determine domain vocabularies in domain analysis
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer = SnowballStemmer(language = 'english')
stem_list = training_syntax['excerpt_clean'][1].split()
new_syntax = ''
for stem in stem_list:
    print("This is an example of the stem of the words within the text/ for debug purpose:  "+ stem + '-->' + SnowballStemmer.stem(stem)) 
    
#create a function to split the language units in the text data and extract their stem
def stemOf_lexUnit(syntax):
    snowball_stemmer = SnowballStemmer(language='english')
    token_list = syntax.split()
    new_syntax = ''
    for token_unit in token_list:
        new_syntax = new_syntax + ' ' + snowball_stemmer.stem(token_unit)
    return new_syntax

#apply the created function above to extract the stem of the lexical units in the unstructured language data
from nltk.stem.snowball import SnowballStemmer
training_syntax['excerpt_clean'] = training_syntax['excerpt_clean'].apply(stemOf_lexUnit)
testing_syntax['excerpt_clean'] = testing_syntax['excerpt_clean'].apply(stemOf_lexUnit)
print("\n")

print("Dictionary Keys Container", len(dictionaryKeys_container.keys()))
print("This is an example of clean texts for debug purpose: ","------>",training_syntax['excerpt_clean'].head(5))

This is an example of the stem of the words within the text/ for debug purpose:  dinner-->dinner
This is an example of the stem of the words within the text/ for debug purpose:  time-->time
This is an example of the stem of the words within the text/ for debug purpose:  mrs-->mrs
This is an example of the stem of the words within the text/ for debug purpose:  fayre-->fayr
This is an example of the stem of the words within the text/ for debug purpose:  somewhat-->somewhat
This is an example of the stem of the words within the text/ for debug purpose:  silent-->silent
This is an example of the stem of the words within the text/ for debug purpose:  eyes-->eye
This is an example of the stem of the words within the text/ for debug purpose:  resting-->rest
This is an example of the stem of the words within the text/ for debug purpose:  dolly-->dolli
This is an example of the stem of the words within the text/ for debug purpose:  wistful-->wist
This is an example of the stem of the words with

 # Assigning parameters and data sets 

In [6]:
#define X and y features
X = training_syntax['excerpt_clean']
y = training_syntax['target']
print("length of X_excerpt_clean: ", len(X),'\n      ',"length of y_target: ", len(y))

#assign a variable for each of the test and train data
test_trained_X = testing_syntax['excerpt_clean']
print("length of test_trained_X:  ",len(test_trained_X))
syntax = training_syntax['excerpt_clean']
print("length of syntax:          ",len(syntax))

#define parametres
lexicalUnit_quantity = 16662
attaching_dimention = 64
max_length = 50
trimming_type = 'post'
packing_type = 'post'
fill_gap = '<OOV>'

length of X_excerpt_clean:  2834 
       length of y_target:  2834
length of test_trained_X:   7
length of syntax:           2834


# Vectorize a text corpus and create pad sequences for both train data and test data

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#vectorize a text corpus, by turning each text into either a sequence of integers
tokenize_engine = Tokenizer(num_words = lexicalUnit_quantity,
                            oov_token=fill_gap)
#Update internal vocabulary based on a list of sequences.
tokenize_engine.fit_on_texts(syntax)
#named list mapping words to their rank/index (int)
lexUnit_index = tokenize_engine.word_index

#Transform each text in texts to a sequence of integers.
textsToSequences_train = tokenize_engine.texts_to_sequences(syntax)
#Making all sequences in a list have the same length: By default this is done by padding 0 in the beginning of each sequence until each sequence has the same length as the longest sequence
pad_sequences = pad_sequences(textsToSequences_train, 
                              maxlen = max_length,
                              truncating=trimming_type,
                              padding = packing_type)
#Creating target label converting the training set into an array 
targetLabel = np.array(training_syntax['target'])



print("This is pad seuqneces of training set:  ",'\n', pad_sequences, '\n\n') 
print("This is a shape of the pad seuqneces of training set:  ",'\n',pad_sequences.shape)
# print("This is a shape of the pad seuqneces of testing set:  ",'\n, pad_sequences_test.shape )
print('\n') 
                           ##############                  ##############                  ##############                  ##############                  ##############                  
# textsToSequences_testing = tokenize_engine.texts_to_sequences(testing_syntax['excerpt_clean'])
# pad_sequences_test = pad_sequences(textsToSequences_testing, 
#                                    maxlen = max_length, 
#                                    trimming_type='post',
#                                    padding = packing_type)

This is pad seuqneces of training set:   
 [[   75     7   189 ...   102    96   270]
 [  576     4   319 ... 10490  2637  2638]
 [ 3097  1491   553 ...   777     5   123]
 ...
 [  177    37   131 ...   330  1870   739]
 [  637   443   585 ...    26  1419   320]
 [   60    73   121 ...     5  1487  1061]] 


This is a shape of the pad seuqneces of training set:   
 (2834, 50)




# Applying LinearRegression & GradientBoostingRegressor MODELS of ML

In [8]:
#################################################################____LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

MSE="Mean Squared Error:"
RMSE="Root Mean Squared Error:"

#split the train and test data sets as categorizing them as traina nd validation types for each x and Y
from sklearn.model_selection import train_test_split
X_train,X_validation,y_train,y_validation = train_test_split(pad_sequences,
                                                             targetLabel,
                                                             random_state = 42)
#Let's model the relationship between two variables by fitting(calculating an equation that minimizes the distance between the fitted line and all of the data points) a linear equation to observed data
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(X_train,y_train)
#predict (the average) Y data from X data
y_prediction = model_LinearRegression.predict(X_validation)
print(MSE, mse(y_validation,
               y_prediction))
print(RMSE, np.sqrt(mse(y_validation,
                        y_prediction)))
print('\n')
#optimization of arbitrary differentiable loss functions: additive model in a forward stage-wise way where in each stage a regression tree is fit on the negative gradient of the given loss function
modelMain_GradientBoostingRegressor = GradientBoostingRegressor(random_state = 42, 
                                                                max_depth = 13, 
                                                                max_features = 19,
                                                                min_samples_split = 600, 
                                                                n_estimators = 70,
                                                                subsample=0.75)
#use fit function of the GradientBoostingRegressor so that to adjust weights according to data values so that better accuracy can be achieved.
modelMain_GradientBoostingRegressor.fit(X_train,y_train)
y_prediction = modelMain_GradientBoostingRegressor.predict(X_validation)
print(MSE, mse(y_validation,
               y_prediction))
print(RMSE, np.sqrt(mse(y_validation,
                        y_prediction)))
print('\n')

Mean Squared Error: 0.854993120472505
Root Mean Squared Error: 0.9246583804154402


Mean Squared Error: 0.7799577599942297
Root Mean Squared Error: 0.8831521726147933




# Obtaining vector representations of words(using GloVe) and create a vector of the syntax

In [9]:
#Applying multilayered bidirectional LSTM to classify text and GloVe embeddings for words(rather than random embeddings)
#GloVe is an unsupervised learning algorithm for obtaining vector representations for words(Training is performed on aggregated global word-word co-occurrence statistics from a corpus, and the resulting representations showcase interesting linear substructures of the word vector space.)
attaching_index = {}
GLoVE_LSTM_dir = '../input/glove6b50dtxt/glove.6B.50d.txt'
GLoVE_LSTM_dir = open(GLoVE_LSTM_dir)
for row in GLoVE_LSTM_dir:
    split_row = row.split()
    lexUnit = split_row[0]
    coefficients= np.asarray(split_row[1:], dtype='float32')
    attaching_index[lexUnit] = coefficients
GLoVE_LSTM_dir.close()
print("this is a debug msg to check the index of the specific word", "\n", "index num of thie word is: ", attaching_index['apple'].shape)
print('\n')

#Creating grid of values(all of the same type) by the help of np.array and multidimensional container of items of the same type and size by the help of np.ndarray.
#sum of vectors are devided by squareRoot
def syntaxVector(s):
    lexUnit_bag = str(s).lower()
    lexUnit_bag = word_tokenize(lexUnit_bag)
    lexUnit_bag = [lexUnit for lexUnit in lexUnit_bag if not lexUnit in stop_words]
    lexUnit_bag = [lexUnit for lexUnit in lexUnit_bag if lexUnit.isalpha()]
    vector = []
    for lexUnit in lexUnit_bag:
        try:
            vector.append(attaching_index[lexUnit])
        except:
            continue
    vector = np.array(vector)
    vector_sum = vector.sum(axis=0)
    if type(vector_sum) != np.ndarray:
        return np.zeros(100)
    return vector_sum / np.sqrt((vector_sum ** 2).sum())

#split the train and test data sets as categorizing them as train and validation types for each x and Y
X_train,X_validation,y_train,y_validation = train_test_split(training_syntax['excerpt_clean'],
                                                             training_syntax['target'],
                                                             test_size = 0.2,random_state = 35)
#Loop through each of the train and validation categories of X data
vectorOf_X_trian = [syntaxVector(trainUnit) for trainUnit in X_train]
vectorOf_X_validation = [syntaxVector(validationUnit) for validationUnit in X_validation]
vectorOf_X_test = [syntaxVector(testUnit) for testUnit in test_trained_X]

this is a debug msg to check the index of the specific word 
 index num of thie word is:  (50,)




# Searching hyperParametres in a randomized manner

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
#define a model
model_GradientBoostingRegressor= GradientBoostingRegressor()
#assign parametres for RandomizedSearchCV
parametres_GBR = {'n_estimators': range(40,160,20), 
                  'max_depth':range(7,22,3), 
                  'min_samples_split':range(180,1020,300),
                  'max_features':range(5,18,1),
                  'subsample':[0.9,0.8,0.80,0.7,0.75,0.7]}

#Searching hyperParametres in a randomized manner implementing fit method 
RandomizedSearch_CV = RandomizedSearchCV(model_GradientBoostingRegressor,
                                         parametres_GBR, cv = 15,
                                         scoring = 'neg_mean_squared_error',verbose=2)
RandomizedSearch_CV.fit(vectorOf_X_trian,y_train)
#Searching hyperParametres in a randomized manner implementing score method
RandomizedSearch_CV1 = RandomizedSearchCV(model_GradientBoostingRegressor,
                                          parametres_GBR, 
                                          cv = 5,
                                          scoring = 'neg_mean_squared_error',
                                          verbose=1)

print('\n')
print('\n')
for i in RandomizedSearch_CV.best_params_:
    print("These are the best solution for my model: ",i)#The drawback of random search(compared ith gridSearch) is that it yields high variance during computing.
print('\n')
print("These are the best estimetor for my model: ",RandomizedSearch_CV.best_estimator_)
print('\n')

finalModel_RandomizedSearch_CV = RandomizedSearch_CV.best_estimator_
finalModel_RandomizedSearch_CV.fit(vectorOf_X_trian,y_train)
y_prediction = finalModel_RandomizedSearch_CV.predict(vectorOf_X_validation)
print(MSE, mse(y_validation,y_prediction))
print(RMSE, np.sqrt(mse(y_validation,y_prediction)))


Fitting 15 folds for each of 10 candidates, totalling 150 fits
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.5s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.5s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.5s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.6s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.6s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.6s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.6s
[CV] END max_depth=13, max_features=17, min_samples_split=480, n_estimators=120, subsample=0.7; total time=   1.5s
[CV] END max_dept

In [11]:
#Create submission for kaggle
y_prediction=pd.DataFrame(y_prediction) 
# finalModel_RandomizedSearch_CV=pd.DataFrame(finalModel_RandomizedSearch_CV) 
# #Save submission to CSV
y_prediction.to_csv('submission.csv', index=False)  
# finalModel_RandomizedSearch_CV.to_csv('finalModel_RandomizedSearch_CV.csv', index=False)  

I would appreciate your upwote. Thanks in advance)))

If you have further questions related to this project, please feel free to contact with me. Email: hosiyatsabirova@gmail.com